#Homework 1

In this HW,  you will create bitcoin transactions.

Lets first install **python-bitcoinlib** which we will use to connect to the Bitcoin network and to generate a public-private key pair.

In [1]:
!pip install python-bitcoinlib
!pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 1.6 MB/s eta 0:00:00


Lets create a Bitcoin address and accompanying private key for ourselves with the code below.

In [2]:
from os import urandom
from bitcoin import SelectParams
from bitcoin.wallet import CBitcoinSecret, P2PKHBitcoinAddress

SelectParams('testnet')

seckey = CBitcoinSecret.from_secret_bytes(urandom(32))
#cPt9QpLmQ3f3iUXFH1ozrcCWnmJ9g1jmFs1ehf6rbvi513m4Ss4Z send back to: tb1qerzrlxcfu24davlur5sqmgzzgsal6wusda40er faucet

print("Private key: %s" % seckey)
print("Address: %s" %P2PKHBitcoinAddress.from_pubkey(seckey.pub))

Private key: cPt9QpLmQ3f3iUXFH1ozrcCWnmJ9g1jmFs1ehf6rbvi513m4Ss4Z
Address: mheAPH8vaH6PaeNiiKSFxKDTTr91Si3nkV


Save this information to somewhere. Otherwise, it may dissappear if you  accidentally erase it and get disconnected. You will not be able to access it later.

Now with this address, we can get free testnet coins from the coin faucet. Click [here](https://coinfaucet.eu/en/btc-testnet/) to get some free coins on the *Bitcoin test network*. Note that it is a nice thing to send back the bitcoin's you got to the faucet.

Once you click the button, a transaction, which is a P2PKH transaction, will be generated and sent to the Bitcoin Test network (by the faucet). One of the outputs of this transaction will be for your Public Key Hash. That output will be our main UTXO to spend.

Before continuing for the exercises, lets configure our Python environment with our public and private key pair. This is nothing fancy; we just set some variables' values to be used later in the codes and the exercises in the rest of this notebook.

Note that we will use the same bitcoin address, the one we created above, to correctly configure our environment.

In [18]:
from bitcoin import SelectParams
from bitcoin.base58 import decode
from bitcoin.wallet import CBitcoinAddress, CBitcoinSecret, P2PKHBitcoinAddress

#This says that we will connect to testnet, not the actual Bitcoin network
SelectParams('testnet')

#This is my private key. You need to add yours here
my_private_key = CBitcoinSecret('cPt9QpLmQ3f3iUXFH1ozrcCWnmJ9g1jmFs1ehf6rbvi513m4Ss4Z')

#As you see, we do not explicitly see the public key  anywhere. We do not need to see it.
#The elliptic curve operations are handled in the background. Your private key
#is in fact a scalar which will be multiplied with a point on the curve to obtain
#the public key (note: elliptic curve discrete logarithm problem).
my_public_key = my_private_key.pub

#With P2PKHBitcoinAddress's from_pubkey function, we can obtain our address.
my_address = P2PKHBitcoinAddress.from_pubkey(my_public_key)

#This is the address of the faucet (a Bitcoin Test account) which we will
#send our money back during/after the exercises.
faucet_address = CBitcoinAddress('tb1qerzrlxcfu24davlur5sqmgzzgsal6wusda40er')

print("Private key: %s" % my_private_key)
print("Address: %s" %P2PKHBitcoinAddress.from_pubkey(my_private_key.pub))

Private key: cPt9QpLmQ3f3iUXFH1ozrcCWnmJ9g1jmFs1ehf6rbvi513m4Ss4Z
Address: mheAPH8vaH6PaeNiiKSFxKDTTr91Si3nkV


In [4]:
my_address.to_scriptPubKey()

CScript([OP_DUP, OP_HASH160, x('174d416ebd231d5331a1cda60a694eb808f8720a'), OP_EQUALVERIFY, OP_CHECKSIG])

The first thing we will do will be splitting the UTXO obtained from the faucet into multiple UTXOs. There are  multiple exercises in this notebook, hence we need more than one UTXO to spend. Besides, we may do mistakes, spend an UTXO in an unlockable manner and lose some of our test coins. So it is a good exercise to split the only UTXO we have.  

Lets run the **split_coins** function below with the transaction we obtained from the faucet.  The code is given below: You do not need to change the **split_coins** function. However, you need to add the UTXO information we obtained from the faucet at the end.

You can check the status of the outputs in the original faucet transaction from BlockCypher (as I do it [here](https://live.blockcypher.com/btc-testnet/tx/1adee36659bc7268d33b3d90ebc7c7cf807f363f11a572076b3a3aa9845342d1/)). Before running the code below it must be *unspent*. Read the code and try to understand what it does by matching what we have learned in the class with the statements below.

If you do everything correctly, at the end, you need to get a response with code 201. That is, you will see something like
```
201 Created
{
  "tx": {
        ....
  }
}
 ```
at the beginning of the output. And after some time, depending on the fee you give to the miners, the UTXO you used will appear as spent. Fortunately,  you have many more now to be used in the exercises.

In [14]:
import requests

from bitcoin.core import b2x, lx, COIN, COutPoint, CMutableTxOut, CMutableTxIn, CMutableTransaction, Hash160
from bitcoin.core.script import *
from bitcoin.core.scripteval import VerifyScript, SCRIPT_VERIFY_P2SH

def create_OP_CHECKSIG_signature(tx, txin_scriptPubKey, seckey):
    #We want no parts of the transaction to be changed.
    #Hence, we hash all the transaction before signing it (i.e.: SIGHASH_ALL).
    sighash = SignatureHash(txin_scriptPubKey, tx, 0, SIGHASH_ALL)
    signature = seckey.sign(sighash) + bytes([SIGHASH_ALL])
    return signature

def broadcast_transaction(tx):
    raw_transaction = b2x(tx.serialize())
    headers = {'content-type': 'application/x-www-form-urlencoded'}
    return requests.post(
        'https://api.blockcypher.com/v1/btc/test3/txs/push',
        headers=headers,
        data='{"tx": "%s"}' % raw_transaction)

#What we simply do in this transaction is splitting. We spent the faucet output and
#split the amount into n pieces each is again to our addresses in a P2PKH structure.
def split_coins(amount_to_send, txid_to_spend, utxo_index, n):
    #We are creating a transaction with a single input and n outputs.

    #txin is the input part of the transaction
    txin_scriptPubKey = my_address.to_scriptPubKey() #initial UTXO was for my address so we need to use it as scriptPubKey
    txin = CMutableTxIn(COutPoint(lx(txid_to_spend), utxo_index)) #now we have txin
    #The function lx() onverts a little-endian hex string to bytes
    #The COutPoint is the combination of a transaction hash and an index n into its vout as we have discussed in the lecture

    #This transaction's outputs are all for myself - so the default P2PKH script
    #[OP_DUP OP_HASH160 <pubKeyHash> OP_EQUALVERIFY OP_CHECKSIG] must be the scriptPubKey
    #The function to_scriptPubKey automatically generates this script from a P2PKHBitcoinAddress
    #Note that the script contains a base16 encoded version of checksummed/modified byte58 encoded address (This operation is reversible!).
    txout_scriptPubKey = my_address.to_scriptPubKey()

    #txout is a single output. (IMP: the function CScript which you use for the first time serializes the script created above)
    txout = CMutableTxOut((amount_to_send / n) * COIN, CScript(txout_scriptPubKey)) #amount and address are given here

    #This is the whole transaction we will broadcast
    #Note the [txout]*n in the parameters of CMutableTransaction function
    tx = CMutableTransaction([txin], [txout] * n)

    #Now lets sign the transaction to keep its integrity
    sig = create_OP_CHECKSIG_signature(tx, txin_scriptPubKey, my_private_key)
    #We need to create the unlocking script which is [signature, encoded SIGHASH_ALL] + PublicKey
    txin.scriptSig = CScript([sig, my_public_key])

    #Lets validate if the script is correctly generated...
    #This function raises a ValidationError subclass if the validation fails
    VerifyScript(txin.scriptSig, txin_scriptPubKey, tx, 0, (SCRIPT_VERIFY_P2SH,))

    #Broadcast the transaction
    response = broadcast_transaction(tx)
    print(response.status_code, response.reason)
    print(response.text)

#The mount of BTC in the output you're splitting minus fee.
#Warning: In the test network, fees are required. They can be small but since this
#is an exercise, you can be generous
# 26.32850582 BTC
# FEES 0.00000147 BTC
# 0.00039388 BTC in address
amount_to_send = 0.00039;

#This must be the transaction id you have from the faucet's transaction
txid_to_spend = ('5de7ad9ca7f3b44b8fe7256e4f398b111ea8f1c03fbb0e56971793eb91afb4a4')

#the output UTXO location in the transaction from faucet (mine is 1, but maybe yours is 0)
utxo_index = 1;

#number of transactions to be splitted
n = 5;
######################################################################

split_coins(amount_to_send, txid_to_spend, utxo_index, n)

201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "8f54e3fcce8d7621e1b3c62532b05dbeb7a587d7352aaaf45f6d64fc12d15821",
    "addresses": [
      "mheAPH8vaH6PaeNiiKSFxKDTTr91Si3nkV"
    ],
    "total": 39000,
    "fees": 388,
    "size": 327,
    "vsize": 327,
    "preference": "low",
    "relayed_by": "34.86.73.98",
    "received": "2024-04-04T10:55:27.809542574Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 5,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "5de7ad9ca7f3b44b8fe7256e4f398b111ea8f1c03fbb0e56971793eb91afb4a4",
        "output_index": 1,
        "script": "47304402206e41baa0718759f1d4b0fe3685a9c52d50f848de5d1064713de02e45eb1a72e7022042c1ccfa1de3311b7a1b1b940751fd01995f627e77e04058864b05451e0024ea0121026e9bbeb0df9890b1e6d060412d5455ea1a1e31a2cf0b937a1691da849135511c",
        "output_value": 39388,
        "sequence": 4294967295,
        "addresses": [
          "mheAPH8vaH6PaeNiiKSFxKDTTr91S

##Part 1 - Exactly same with the given practice exercise

Here is the first part: it is about spending one of the smaller transactions and send it back to the **faucet**. Make it complete, run, and be sure that you have the correct output with code *201 Created*.

**Hint**:  Almost all the information you need, the codes, functions to be used etc. can be read and copied from the **split_coins** example above. Before, we created a transaction to split the original UTXO obtained from the faucet. You will do the same here but this time you will not direct the output to yourself. The money needs to go back to the faucet.

In [21]:
from bitcoin.core.script import *

# send back to: tb1qerzrlxcfu24davlur5sqmgzzgsal6wusda40er faucet
# TODO: Complete this script to unlock the BTC that was sent to you
# in the PayToPublicKeyHash transaction. You may need to use variables
# that are globally defined.
def send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index, txout_scriptPubKey):
    #line 1: create the txin below
    txin = CMutableTxIn(COutPoint(lx(txid_to_spend), utxo_index))

    #line 2: create the txout below
    txout = CMutableTxOut((amount_to_send / n) * COIN, CScript(txout_scriptPubKey))

    #line 3: create the tx below
    tx = CMutableTransaction([txin], [txout] * n)

    #line 4: create the txin_scriptPubKey
    txin_scriptPubKey = my_address.to_scriptPubKey()

    #line 5: create the signature by using the appropriate function above
    sig = create_OP_CHECKSIG_signature(tx, txin_scriptPubKey, my_private_key)

    #line 6: set the txin's scriptSig value (signature + public_key)

    txin.scriptSig = CScript([sig, my_public_key])

    #verifying the transaction (this should pass)
    VerifyScript(txin.scriptSig, txin_scriptPubKey, tx, 0, (SCRIPT_VERIFY_P2SH,))

    return broadcast_transaction(tx)

######################################################################
# TODO: set all these parameters correctly
amount_to_send = 0.0000000078 #Do not forget the fee
txid_to_spend = ('9fc024992a9dbabd35364184bb706e3953419727c03d389d202fc5f7e01d0b4b')
utxo_index = 1
txout_scriptPubKey = faucet_address.to_scriptPubKey() #[OP_DUP ....]
######################################################################

response = send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index, txout_scriptPubKey)
print(response.status_code, response.reason)
print(response.text)

201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "764b554df8418619ec0635b11d49b38f5889fc7cdc7d5312f53fb185b2335dd2",
    "addresses": [
      "mheAPH8vaH6PaeNiiKSFxKDTTr91Si3nkV",
      "tb1qerzrlxcfu24davlur5sqmgzzgsal6wusda40er"
    ],
    "total": 0,
    "fees": 15,
    "size": 312,
    "vsize": 312,
    "preference": "low",
    "relayed_by": "34.86.73.98",
    "received": "2024-04-04T12:11:44.789020774Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 5,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "9fc024992a9dbabd35364184bb706e3953419727c03d389d202fc5f7e01d0b4b",
        "output_index": 1,
        "script": "473044022073e73a574c99385c79070566cf991cb1b23c9c0fa97c47817cba4ea5fd6a9ecf022036264647a8663177f4235a956c4412ae21390c1011e916cda8ac0bbdaaf551c70121026e9bbeb0df9890b1e6d060412d5455ea1a1e31a2cf0b937a1691da849135511c",
        "output_value": 15,
        "sequence": 4294967295,
        "addresses"

#Part 2 - The puzzle is different than the one in the exercise

In this part, you will create a UTXO and later redeem it by using P2SH approach. The locking script will contain the hash of a puzzle whose solution will be given in the redeem script.

To unlock, the locking script will require three numbers whose sum is 15 and maximum is smaller than 8. Your script must check these equalities programatically.

For the opcodes you may need to use please check this [webpage](https://en.bitcoin.it/wiki/Script). Again if you are successful, you will get the return code **201** after the transaction broadcast.

You will complete this exercise in two parts. First a transaction will be created with a single output. Then it will be redeemed (by youself, to be sent to the faucet). The starting code for the first part is given below:



In [101]:
from sys import exit
from bitcoin.core.script import *
from bitcoin.core.script import CScript
from bitcoin.wallet import CBitcoinAddress, P2SHBitcoinAddress
from bitcoin.core import x, b2lx, lx, Hash160

######################################################################
# TODO: Complete the scriptPubKey implementation with operations


# To unlock, the locking script will require three numbers whose sum is 15 and maximum is smaller than 8. Your script must check these equalities programatically.
locking_script = [
     # placeholders
    OP_1, OP_2, OP_3,
    OP_DUP, OP_DUP,  # duplicate first two numbers on top of stack to check
    OP_8, OP_LESSTHAN,  # check no 1 <8
    OP_DUP, OP_DUP,    # no 2 < 8
    OP_8, OP_LESSTHAN,
    OP_ROT,
    OP_8, OP_LESSTHAN,  # no 3 < 8

    OP_BOOLAND,
    OP_BOOLAND,

    # sum of the three numbers
    OP_ROT, OP_ADD,  # add  two numbers
    OP_ROT, OP_ADD,  # add the result to the third number
    OP_15, OP_EQUALVERIFY, # Check if the sum equals 15
    OP_1
]

#create lock script hash to script out P2SH
locking_script = CScript(locking_script)
locking_script_hash = Hash160(locking_script)

# Be careful while using OP_EQUAL and OP_VERIFY - read them from https://en.bitcoin.it/wiki/Script
ex2a_txout_scriptPubKey = [ OP_HASH160,b'locking_script_hash', OP_EQUAL]
######################################################################

######################################################################
# TODO: set these parameters correctly
amount_to_send = 0.000000015
txid_to_spend = ("8f54e3fcce8d7621e1b3c62532b05dbeb7a587d7352aaaf45f6d64fc12d15821")
utxo_index = 3
######################################################################

#This function is from the first exercise
response = send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index, ex2a_txout_scriptPubKey)
print(response.status_code, response.reason)
print(response.text)

409 Conflict
{}


In the second part, we will redeem by only providing the solution of the locking script. Note that anyone can do this. Since the lock script does not contain any signature, address etc. check, one only need to provide **a**, **b** and **c**. You can follow the results, e.g., confirmations, from BlockCypher test network explorer by using the transaction hash you obtained after succesfully completing this part.

In [103]:
def send_from_custom_transaction(amount_to_send, txid_to_spend, utxo_index, txin_scriptPubKey, txin_scriptSig, txout_scriptPubKey):
  txout = CMutableTxOut((amount_to_send / n) * COIN, CScript(txout_scriptPubKey))
  txin = CMutableTxIn(COutPoint(lx(txid_to_spend), utxo_index))
  tx = CMutableTransaction([txin], [txout] * n)
  txin.scriptSig = CScript(txin_scriptSig)


    #Complete this function, if you have implemented send_from_P2PKH_transaction
    #the steps are the same. The main difference is the scripts are given as parameters
    #since you will define them below.
  VerifyScript(txin.scriptSig, CScript(txin_scriptPubKey), tx, 0, (SCRIPT_VERIFY_P2SH,))

  return broadcast_transaction(tx)

######################################################################
# TODO: set these parameters correctly
amount_to_send = 0.000000078
txid_to_spend = '8f54e3fcce8d7621e1b3c62532b05dbeb7a587d7352aaaf45f6d64fc12d15821'
utxo_index = 3
n = 5
######################################################################
"""
<arg1> <arg2>... <serializedScript>
OP_HASH160 <scriptHash> OP_EQUAL
"""


txin_scriptPubKey = ex2a_txout_scriptPubKey
######################################################################
# TODO: implement the scriptSig for redeeming the transaction created in  Exercise 2a.
# This must be a very chort scriptSig. No signature, address check etc. script that fulfills the locking conditions!!
# Anyone who can submit the solution of the problem (a and b in correct form) can spend the transaction
faucet_address = CBitcoinAddress('tb1qerzrlxcfu24davlur5sqmgzzgsal6wusda40er')
######################################################################
txout_scriptPubKey = faucet_address.to_scriptPubKey() #sending money to faucet after solving the puzzle

response = send_from_custom_transaction(amount_to_send, txid_to_spend, utxo_index, txin_scriptPubKey, txin_scriptSig, txout_scriptPubKey)
print(response.status_code, response.reason)
print(response.text)

VerifyScriptError: scriptPubKey returned false

##Part 3: Slightly different than the practice

In the first part of the third exercise, you will create and broadcast a MULTISIG transaction to the test network. The second part will be about providing an unlock script for this transaction. You will definitely want to read how OP_CHECKMULTISIG works from [here](https://en.bitcoin.it/wiki/Script).

The scenario is as follows: in a company, there is a manager and three employees. The multisig signature
you will create (in the first part) can be redeemed by the manager combined with one of the employees. However, only the manager, or only the employees with enough number of signatures will not be able to redeem the transaction. **Unlike the HW, the order of the signatures should not matter**.

You may assume the role of the manager for this exercise so that the manager’s private key is your private key and the manager’s public key is your public key.

For the employee private keys and addresses, you can use the key generation code at the beginning of this notebook. That is generate 3 random Bitcoin secret keys to be used in this exercise with the code below.


In [36]:
from os import urandom
from bitcoin import SelectParams
from bitcoin.wallet import CBitcoinSecret, P2PKHBitcoinAddress

SelectParams('testnet')

seckey = CBitcoinSecret.from_secret_bytes(urandom(32))
"""
employee 1
Private key: cTPArq2qqkPQpsYBnrJ3c1GUCqgRVnXd9ghi2tz71wamGNdUARyZ
Address: mmJVhJLsGjR1UsoaSqhCRbhM161Ju2o26Y
employee 2
Private key: cPtDFPWefDDx516zSsSccEopSrV1rs3sY3K4QL7uMMFHi6ENZGMD
Address: n1AxyA8XdAG19AzEgmQmAnJnSexFQkWoqY
employee 3
Private key: cRgEMFSVY2B1P9faPHcULeJdHjLP2z5F7mXhDtwrePf81bzLRat4
Address: mgyPEYd2AL4DHiS4774T5fh6MQ7YGonMCU
"""
print("Private key: %s" % seckey)
print("Address: %s" %P2PKHBitcoinAddress.from_pubkey(seckey.pub))

Private key: cRgEMFSVY2B1P9faPHcULeJdHjLP2z5F7mXhDtwrePf81bzLRat4
Address: mgyPEYd2AL4DHiS4774T5fh6MQ7YGonMCU


After filling the spaces ---xxx-- in the first part below, we can create the transaction locked as described above.

In [78]:
from sys import exit
from bitcoin.core.script import *

#Run the key generation code at the beginning of this notebook multiple times
#to create private keys and addresses for the employees.
employee1_private_key = CBitcoinSecret('cTPArq2qqkPQpsYBnrJ3c1GUCqgRVnXd9ghi2tz71wamGNdUARyZ')
employee1_public_key = employee1_private_key.pub
employee2_private_key = CBitcoinSecret('cPtDFPWefDDx516zSsSccEopSrV1rs3sY3K4QL7uMMFHi6ENZGMD')
employee2_public_key = employee2_private_key.pub
employee3_private_key = CBitcoinSecret('cRgEMFSVY2B1P9faPHcULeJdHjLP2z5F7mXhDtwrePf81bzLRat4')
employee3_public_key = employee3_private_key.pub

######################################################################
# TODO: Complete the scriptPubKey implementation for Exercise 3

# You can assume the role of the manager for the purposes of this problem
# and use my_public_key and my_private_key in lieu of manager_public_key and

manager_public_key = my_public_key
manager_private_key = my_private_key
# manager_private_key.

ex3a_txout_scriptPubKey = [OP_0,
                           # any can sign
                           employee1_public_key,
                           employee2_public_key,
                           employee3_public_key,
                           manager_public_key,
                           OP_CHECKMULTISIGVERIFY
                           ]
######################################################################

######################################################################
# TODO: set these parameters correctly
amount_to_send = 0.0000000077
txid_to_spend = ('9fc024992a9dbabd35364184bb706e3953419727c03d389d202fc5f7e01d0b4b')
utxo_index = 4
######################################################################

response = send_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_index, ex3a_txout_scriptPubKey)
print(response.status_code, response.reason)
print(response.text)

201 Created
{
  "tx": {
    "block_height": -1,
    "block_index": -1,
    "hash": "eb84beb9959a577267bfb88062f5172f3a5941d917187820ce6bba2fda8fe653",
    "addresses": [
      "mheAPH8vaH6PaeNiiKSFxKDTTr91Si3nkV"
    ],
    "total": 0,
    "fees": 15,
    "size": 893,
    "vsize": 893,
    "preference": "low",
    "relayed_by": "34.86.73.98",
    "received": "2024-04-04T18:23:12.609798321Z",
    "ver": 1,
    "double_spend": false,
    "vin_sz": 1,
    "vout_sz": 5,
    "confirmations": 0,
    "inputs": [
      {
        "prev_hash": "9fc024992a9dbabd35364184bb706e3953419727c03d389d202fc5f7e01d0b4b",
        "output_index": 4,
        "script": "483045022100d91cfac1a0bb8f7479335698ee34d01dde91684ebc8a1176e36dadd81cb6244b022072978c9bf4c20f92e99b53018e6f1a08d7264df37935aba7e8afe76ef46cfcd80121026e9bbeb0df9890b1e6d060412d5455ea1a1e31a2cf0b937a1691da849135511c",
        "output_value": 15,
        "sequence": 4294967295,
        "addresses": [
          "mheAPH8vaH6PaeNiiKSFxKDTTr91Si3nkV"


In the second part, you will redeem the transaction. Keep scriptPubKey  as small as you can. One can use any legal combination of signatures to redeem the transaction but make sure that all combinations would have worked.

In [108]:
from sys import exit
from bitcoin.core.script import *

employee_IDs = [1,2,3]
#emplfromoyee ID is 1,2 or 3
def multisig_scriptSig(tx, txin_scriptPubKey, employee_ID):
    manager_sig = create_OP_CHECKSIG_signature(tx, txin_scriptPubKey, my_private_key)
    if employee_ID == 1:
      employee_sig = create_OP_CHECKSIG_signature(tx, txin_scriptPubKey, employee1_private_key)
    elif employee_ID == 2:
      employee_sig = create_OP_CHECKSIG_signature(tx, txin_scriptPubKey, employee2_private_key)
    else:
      employee_sig = create_OP_CHECKSIG_signature(tx, txin_scriptPubKey, employee3_private_key)
    ######################################################################
    # TODO: Complete this script to unlock the BTC that was locked in the multisig transaction created in Exercise 3a.
    return [ OP_0,
        manager_sig,
        employee_sig, OP_CHECKMULTISIGVERIFY]
    ######################################################################
employee_IDs = [1,2,3]

def send_from_multisig_transaction(amount_to_send, txid_to_spend, utxo_index, txin_scriptPubKey, txout_scriptPubKey):
    #Complete this function, if you have implemented send_from_P2PKH_transaction
    #the steps are the same. The main difference is the scripts are given as parameters
    #since you will define them below.
    for employee_ID in employee_IDs:
      txin = CMutableTxIn(COutPoint(lx(txid_to_spend), utxo_index))

      txout = CMutableTxOut(amount_to_send, txout_scriptPubKey)

      tx = CMutableTransaction([txin], [txout])

      #create scriptSig - note that this is different than before (use the multisig_scriptSig function above)
      txin.scriptSig = CScript(multisig_scriptSig(tx, txin_scriptPubKey, employee_ID))


      VerifyScript(txin.scriptSig, CScript(txin_scriptPubKey), tx, 0, (SCRIPT_VERIFY_P2SH,))

    return broadcast_transaction(tx)

######################################################################
# TODO: set these parameters correctly
amount_to_send = 0.00000005
txid_to_spend = ('9fc024992a9dbabd35364184bb706e3953419727c03d389d202fc5f7e01d0b4b')
utxo_index = 3
######################################################################

txin_scriptPubKey = ex3a_txout_scriptPubKey
txout_scriptPubKey = faucet_address.to_scriptPubKey()

response = send_from_multisig_transaction(amount_to_send, txid_to_spend, utxo_index, txin_scriptPubKey, txout_scriptPubKey)
print(response.status_code, response.reason)
print(response.text)

AttributeError: 'list' object has no attribute 'is_witness_scriptpubkey'

##Part 4: This is the same with the practice

Redirect all remaining UTXOs to faucet

In [114]:
from bitcoin.core.script import *

def send_multiple_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_indexes, txout_scriptPubKey):
    txins = [CMutableTxIn(COutPoint(lx(txid_to_spend), utxo_index)) for utxo_index in utxo_indexes];
    #Fill the txin list with the remaining transactions (txins)
      #you may use a for loop here so there can be a number of statements

    #You will have a single txout create it (txout)
    txout = CMutableTxOut(amount_to_send, txout_scriptPubKey)
    #Create the transaction (tx)
    tx = CMutableTransaction(txins, [txout])

    #the function below creates the signature. you do not need to modify it
    txin_scriptPubKey = my_address.to_scriptPubKey()
    for input_index, utxo_index in enumerate(utxo_indexes):
      sighash = SignatureHash(txin_scriptPubKey, tx, input_index, SIGHASH_ALL)
      signature = seckey.sign(sighash) + bytes([SIGHASH_ALL])
      txins[input_index].scriptSig = CScript([signature, my_public_key])
      VerifyScript(txins[input_index].scriptSig, txin_scriptPubKey, tx, input_index, (SCRIPT_VERIFY_P2SH,))
    return broadcast_transaction(tx)

######################################################################
# TODO: set all these parameters correctly
amount_to_send = 0.00000015  #Do not forget the fee
txid_to_spend = ('764b554df8418619ec0635b11d49b38f5889fc7cdc7d5312f53fb185b2335dd2')
utxo_indexes = [0,1,2,3,4]
txout_scriptPubKey = faucet_address.to_scriptPubKey() #[OP_DUP ....]
######################################################################

response = send_multiple_from_P2PKH_transaction(amount_to_send, txid_to_spend, utxo_indexes, txout_scriptPubKey)
print(response.status_code, response.reason)
print(response.text)

VerifyScriptError: scriptPubKey returned false